In [89]:
import os
os.getcwd()

'C:\\Users\\SoumyaRoy\\Downloads'

In [90]:
os.chdir('C:/Users/SoumyaRoy/Downloads')
import pandas as pd
df = pd.read_csv("TrainingData_labels.csv", encoding ='latin1')
df.columns
df.shape
df.head()

,Unnamed: 0,carrier,sentiment,polarity,churn_likeliness,clf_text
0,2750,att tmobile,pos,0.4375,0,@tmobile @att glad to hear working out for you...
1,2765,att tmobile,neu,0.0000,0,@att @tmobile r u a virgin or do you just play...
2,2804,att tmobile,neu,0.0000,1,oh gawd don t do that they have no tower atm n...
3,2892,att tmobile,neu,0.0000,1,on the same note my dad switched his phone int...
4,2897,att tmobile,neu,0.0000,1,switched to @tmobile from @att and i am just b...


In [91]:
df['churn_likeliness'].value_counts()

0    571
1    224
Name: churn_likeliness, dtype: int64

In [98]:
df.shape

(795, 6)

In [92]:
df['clf_text'].isnull().sum()

0

In [84]:
Cleaning text
import time
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

start_time = time.time()                                    
clean_text = []
for text in df['question']:
    words = regexp_tokenize(text.lower(), r'[A-Za-z]+')
    words = [w for w in words if len(w)>1 and w not in stopwords.words('english')]
    words = [lemmatizer.lemmatize(w) for w in words]
    clean_text.append(' '.join(words))
print('Elapsed clock time: ', (time.time() - start_time)/60, ' minutes')

len(clean_text)
df['clean_text'] = clean_text
df.head()

In [93]:
# Split data into train and validation and encode label
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_x, valid_x, train_y, valid_y = \
    train_test_split(df['clf_text'], df['churn_likeliness'], \
    test_size=0.2, random_state=45)

In [95]:
#Converting the y value to numeric
# encoder = LabelEncoder()
# train_y = encoder.fit_transform(train_y)
# valid_y = encoder.fit_transform(valid_y)

In [115]:
#Converting the x value to numeric
#Text vectorization: Create count, TF-IDF, and n-gram vectorizers

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# n-gram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', \
    ngram_range=(1,5), max_features=5000)
tfidf_vect_ngram.fit(df['clf_text'])
xtrain_tfidf_ngram = tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram = tfidf_vect_ngram.transform(valid_x)

In [116]:
# Classification Models and Success metrics
  
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score
  
def model(classifier, train_vector, valid_vector):
    classifier.fit(train_vector, train_y)  
    predict_y = classifier.predict(valid_vector)
    accuracy = accuracy_score(valid_y, predict_y)
    precision = precision_score(valid_y, predict_y,labels=[0,1,2,3,4,5,6],average=None)
    recall = recall_score(valid_y, predict_y,labels=[0,1,2,3,4,5,6],average=None)
    conf_matrix = confusion_matrix(valid_y, classifier.predict(valid_vector))
    print('\nAccuracy_score: {:.3f}'.format(accuracy ))
    print('precision_score: {:.3f}'.format(precision.mean()))
    print('recall_score: {:.3f}'.format(recall.mean()))
    print('\nconfusion_matrix: \n')
    
    print(confusion_matrix(valid_y, classifier.predict(valid_vector)))
    #print(classification_report(valid_y, classifier.predict(valid_vector)))
    return 


# XGBClassifier and Naive Bayes 

In [117]:
from sklearn.naive_bayes import MultinomialNB
model(MultinomialNB(), xtrain_tfidf_ngram, xvalid_tfidf_ngram)


Accuracy_score: 0.698
precision_score: 0.206
recall_score: 0.157

confusion_matrix: 

[[105   2]
 [ 46   6]]


C:\Users\SoumyaRoy\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\SoumyaRoy\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [118]:
from xgboost import XGBClassifier
model(XGBClassifier(), xtrain_tfidf_ngram.tocsc(), xvalid_tfidf_ngram.tocsc())


Accuracy_score: 0.736
precision_score: 0.205
recall_score: 0.182

confusion_matrix: 

[[99  8]
 [34 18]]


In [119]:
m=XGBClassifier()

In [120]:
m.fit(xtrain_tfidf_ngram,train_y)
predict_y = m.predict(xvalid_tfidf_ngram)
predict_y

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0], dtype=int64)

In [121]:
valid_y

67     1
349    1
272    0
124    0
276    0
504    0
702    0
84     1
467    0
642    0
664    0
468    1
701    0
214    0
248    0
65     0
418    1
503    1
422    0
669    0
169    0
713    0
719    0
154    1
611    0
357    1
339    1
30     1
285    0
66     0
      ..
476    1
204    0
290    1
26     0
353    0
510    0
792    0
382    1
343    0
409    1
208    0
520    0
660    0
633    0
482    0
111    1
206    0
419    0
484    0
480    0
372    1
630    0
471    0
605    0
256    0
48     0
515    1
668    1
609    0
329    1
Name: churn_likeliness, Length: 159, dtype: int64

In [122]:
df.iloc[349]

Unnamed: 0                                                      56991
carrier                                                   att tmobile
sentiment                                                         neg
polarity                                                    -0.195238
churn_likeliness                                                    1
clf_text            @att @tmobile smart move leaving @att just get...
Name: 349, dtype: object